In [3]:
import pandas as pd
import glob
import re
from datetime import datetime

## Date Formatting

In [4]:
files = ['Scrapped Data/data/opindia.xlsx',
 'Scrapped Data/data/boomlive.xlsx',
 'Scrapped Data/data/altnews.xlsx']

for i in files:
    df = pd.read_excel(i)
    print(i)
    print(df.iloc[0].date)

Scrapped Data/data/opindia.xlsx
23 February, 2024 
Scrapped Data/data/boomlive.xlsx
11 March 2024 8:24 AM GMT
Scrapped Data/data/altnews.xlsx
28th February 2024 / 3:14 pm


In [10]:
f = files[0]
df = pd.read_excel(f)

df.date = df.date.apply(lambda x: x.strip('\n '))
df.date = df.date.apply(lambda x: datetime.strptime(x, '%d %B, %Y').strftime('%B %d, %Y'))
df.date = df.date.apply(lambda x: x.replace(' 0', ' '))
df.to_excel('Cleaned Data/'+f[19:], index=False)

df.head()

,links,date,title,text
0,https://www.opindia.com/2024/02/fact-check-shy...,"February 23, 2024",YouTuber Shyam Meera Singh passes off old imag...,Shyam Meera Singh did not bother to verify the...
1,https://www.opindia.com/2024/02/amethi-dm-says...,"February 21, 2024",Amethi: DM says govt land secured from illegal...,The District Magistrate pointed out that the a...
2,https://www.opindia.com/2024/02/pm-modi-lied-a...,"February 8, 2024",‘PM Modi lied about his OBC caste’ claims Cong...,The trend of incorporating other castes in th...
3,https://www.opindia.com/2024/02/congress-leade...,"February 7, 2024",Congress leader Surendra Rajput spreads fake n...,"In his Rajya Sabha speech, PM Modi cited a let..."
4,https://www.opindia.com/2024/02/nehru-did-not-...,"February 7, 2024",‘Nehru did not call Indians lazy and unintelli...,One is pressed to understand what Upadhyay was...


In [11]:
def conv(s):
    try:
        return datetime.strptime(s, '%d %B %Y %I:%M %p %Z').strftime('%B %d, %Y')
    except:
        return datetime.strptime(s, '%d %b %Y %I:%M %p %Z').strftime('%B %d, %Y')

f = files[1]
df = pd.read_excel(f)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

df.date = df.date.apply(lambda x: x.strip('\n '))
df.date = df.date.apply(conv)
df.date = df.date.apply(lambda x: x.replace(' 0', ' '))
df.to_excel('Cleaned Data/'+f[19:], index=False)

df.head()

,links,date,title,text
0,https://www.boomlive.in/fact-check/man-thrashe...,"March 11, 2024",Video From Punjab Peddled As Christian Man Bea...,BOOM found that there is no communal angle to ...
1,https://www.boomlive.in/fact-check/factcheck-v...,"March 10, 2024",Pak Market Video Peddled Falsely As UK's Bradf...,BOOM found that the viral video is old and was...
2,https://www.boomlive.in/fact-check/viral-photo...,"March 8, 2024",Photo Of Army Officer At Husband's Funeral Wit...,BOOM found that the photograph is from the fun...
3,https://www.boomlive.in/fact-check/tata-memori...,"March 8, 2024",Tata Memorial's ₹100 Pill Misreported As Cance...,BOOM spoke to oncologists who emphasised that ...
4,https://www.boomlive.in/fact-check/mallikarjun...,"March 8, 2024",Cropped Video of Mallikarjun Kharge Saying Con...,BOOM found that the video has been cropped to ...


In [12]:
def conv(s):
    match = re.search(r'(\d{1,2})(?:st|nd|rd|th)\s(\w+)\s(\d{4})', s)
    if match:
        day = match.group(1)
        month = match.group(2)
        year = match.group(3)
    
        month_num = datetime.strptime(month, '%B').month
    
        return datetime(int(year), month_num, int(day)).strftime('%B %d, %Y')
    else:
        return None

f = files[2]
df = pd.read_excel(f)

df.date = df.date.apply(lambda x: x.strip('\n '))
df.date = df.date.apply(conv)
df.date = df.date.apply(lambda x: x.replace(' 0', ' '))
df.to_excel('Cleaned Data/'+f[19:], index=False)

df.head()

,links,date,title,text
0,https://www.altnews.in/modi-effigy-tied-to-a-t...,"February 28, 2024",Modi effigy tied to a tractor: Photo from Oreg...,A video of Prime Minister Narendra Modi’s effi...
1,https://www.altnews.in/bargain-over-tractor-pr...,"February 23, 2024",Bargain over tractor price: Video falsely shar...,A video of an altercation between a group of t...
2,https://www.altnews.in/baps-mandir-inaugurated...,"February 23, 2024",BAPS Mandir inaugurated by Modi is not the fir...,Prime Minister Narendra Modi visited Abu Dhabi...
3,https://www.altnews.in/false-claim-of-muslim-p...,"February 20, 2024",Haldwani violence: False claims about spike in...,"On February 8, violence broke out in Haldwani,..."
4,https://www.altnews.in/edited-video-of-dallewa...,"February 20, 2024",News channels run doctored video of farmer lea...,A video related to the farmers’ movement is vi...


## Text Formatting

In [13]:
files = ['Cleaned Data/opindia.xlsx',
 'Cleaned Data/boomlive.xlsx',
 'Cleaned Data/altnews.xlsx']

def is_within_range(text):
    year = int(text[-4:])
    if 2018 <= year <= 2023:
        return True
    return False

for f in files:
    df = pd.read_excel(f)
    
    df.title = df.title.apply(lambda x: x.strip('\n '))
    df.title = df.title.apply(lambda x: x.replace('\n', ' ').replace('\t', ' '))
    df.title = df.title.apply(lambda x: re.sub(' +', ' ', x))
    
    df.text = df.text.apply(lambda x: x.replace('\n', ' '))

    df.text = df.text.apply(lambda x: re.sub(' +', ' ', x))
    
    df = df[df['date'].apply(is_within_range)]

    df["date_year"] = df["date"].apply(lambda x: x[-4:])
    df["date_month"] = df["date"].apply(lambda x: x.split(",")[0].split(' ')[0])
    df["date_day"] = df["date"].apply(lambda x: x.split(",")[0].split(' ')[1])

    df = df[['links', 'date_year', 'date_month', 'date_day', 'title', 'text']]
        
    df.to_excel(f, index=False)

## Avg article length

In [14]:
def max_avg_length(strings):
    if not strings:
        return 0, 0

    total_length = 0
    max_length = 0
    for string in strings:
        total_length += len(string.split(' '))
        max_length = max(max_length, len(string.split(' ')))

    avg_length = total_length / len(strings)
    return max_length, avg_length

for f in files:
    df = pd.read_excel(f)
    df.text = df.text.apply(lambda x: re.sub(r'([^A-Za-z@.])', ' ', x))
    df.text = df.text.apply(lambda x: re.sub(' +', ' ', x))
    strings = list(df.text)
    max_length, avg_length = max_avg_length(strings)
    print(f)
    print(f"Max length: {max_length}, Average length: {avg_length}\n")


Cleaned Data/opindia.xlsx
Max length: 5412, Average length: 739.3927631578947

Cleaned Data/boomlive.xlsx
Max length: 4806, Average length: 531.0272977710994

Cleaned Data/altnews.xlsx
Max length: 5448, Average length: 532.8193197921587



## Year wise # of articles

In [15]:
for f in files:
    df = pd.read_excel(f)
    print(f)
    print(df.date_year.value_counts(),end="\n\n")

Cleaned Data/opindia.xlsx
date_year
2019    404
2020    400
2021    260
2022    174
2018    148
2023    134
Name: count, dtype: int64

Cleaned Data/boomlive.xlsx
date_year
2020    841
2023    717
2019    704
2022    701
2021    693
2018    337
Name: count, dtype: int64

Cleaned Data/altnews.xlsx
date_year
2019    1013
2020     987
2021     790
2018     535
2022     474
2023     435
Name: count, dtype: int64

